In [1]:
!pip install tf2onnx
!pip install --upgrade keras_cv
!pip install onnxruntime
!pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/455.8 kB ? eta -:--:--

     ━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━ 184.3/455.8 kB 5.3 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 7.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/15.9 MB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 8.8/15.9 MB 128.6 MB/s eta 0:00:01

...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━ 276.5/311.2 kB 8.3 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import tensorflow as tf
import numpy as np
from joblib import load
import pandas as pd
import copy

import librosa.display
import matplotlib.pyplot as plt

import librosa
import albumentations as A

from joblib import Parallel, delayed

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import LabelBinarizer

import os, gc

from scipy import signal as sci_signal
import torch

import albumentations as albu
import random

from sklearn.model_selection import train_test_split
import cv2
from joblib import load, dump

import tf2onnx
import onnxruntime

In [3]:
x, y = load('/kaggle/input/data-mid-and-start/mid_start_train.joblib')
print(x.shape, y.shape)

(47630, 256, 256, 1) (47630, 182)


In [4]:
class TrainDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, x_data = x, y_data = y, indices = [], batch_size=32, ):
            
        self.x_data = x_data
        self.y_data = y_data
        self.batch_size = batch_size        
        self.indices = np.arange(len(self.x_data))
        

    def __len__(self):
        return int(np.ceil(len(self.x_data) / self.batch_size))
        
    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        x_batch = self.x_data[batch_indices]
        y_batch = self.y_data[batch_indices]        
        x_batch = x_batch.reshape(-1,256,256,1)#128, 313, 510, 512
        
        return x_batch, y_batch    
    
    def on_epoch_end(self):
        pass

In [5]:
gen = TrainDataGenerator()
for i in gen:
    print(i[0].shape, i[1].shape)
    break

(32, 256, 256, 1) (32, 182)


In [6]:
session = onnxruntime.InferenceSession('/kaggle/input/torchvision-spects-models/mid_5sec_256_256.onnx')

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [7]:
preds = []
for i in tqdm(gen):
    preds.append(session.run([output_name], {input_name: i[0].reshape(-1,256,256,1)})[0])
    
preds = np.vstack(preds)
preds.shape

  0%|          | 0/1489 [00:00<?, ?it/s]

  0%|          | 1/1489 [00:00<07:31,  3.30it/s]

  0%|          | 2/1489 [00:00<07:29,  3.31it/s]

  0%|          | 3/1489 [00:00<06:37,  3.73it/s]

  0%|          | 4/1489 [00:01<06:12,  3.99it/s]

...

100%|█████████▉| 1487/1489 [05:32<00:00,  4.48it/s]

100%|█████████▉| 1488/1489 [05:32<00:00,  4.48it/s]

100%|██████████| 1489/1489 [05:32<00:00,  5.21it/s]

100%|██████████| 1489/1489 [05:32<00:00,  4.47it/s]

(47630, 182)

In [8]:
from scipy.stats import entropy

def kl_div(p1, p2):
    '''p2 - labels'''
    p1 = p1+ 1e-10
    p2 = p2+ 1e-10
    return entropy(p1, p2)

In [9]:
dump(preds, 'preds.joblib')

['preds.joblib']

In [10]:
temp = np.sum(y, axis = 1)
print(temp.shape)
mult_label = np.where(temp>1)[0]
print(mult_label.shape)


(47630,)
(3480,)


In [11]:
# for i in mult_label:
#     inds = np.where(y[i]>0)[0]
#     y[i][inds] = 0.2

In [12]:
# cnt = 0
# for i in mult_label:
#     if np.argmax(y[i]) == np.argmax(preds[i]):
#         cnt += 1 
#     else:
#         args1 = np.argsort(y[i])
#         args2 = np.argsort(preds[i])
#         if args2[0] in args1[0:3]:
#             y[i] = y[i]*0.60 + preds[i]*0.40
#         elif args2[1] in args1[0:3]:
#             y[i] = y[i]*0.60 + preds[i]*0.40
#         elif args2[2] in args1[0:3]:
#             y[i] = y[i]*0.60 + preds[i]*0.40

In [13]:
kls = []
for i in tqdm(range(len(y))):
    kls.append(kl_div(preds[i], y[i]))

kls = np.asarray(kls)
print(kls.shape)

  0%|          | 0/47630 [00:00<?, ?it/s]

  0%|          | 192/47630 [00:00<00:24, 1913.39it/s]

  1%|          | 384/47630 [00:00<00:25, 1880.01it/s]

  1%|▏         | 597/47630 [00:00<00:23, 1989.67it/s]

  2%|▏         | 808/47630 [00:00<00:23, 2034.74it/s]

...

 99%|█████████▉| 47199/47630 [00:23<00:00, 2029.86it/s]

100%|█████████▉| 47412/47630 [00:23<00:00, 2056.30it/s]

100%|█████████▉| 47626/47630 [00:23<00:00, 2079.98it/s]

100%|██████████| 47630/47630 [00:23<00:00, 2050.64it/s]

(47630,)


In [14]:
args = np.argsort(kls)

In [15]:
kls[args[36500]]

18.408353997510723

In [16]:
x_temp = x[args[0:36_500]]
print(x_temp.shape)
del x
gc.collect()

(36500, 256, 256, 1)


0

In [17]:
dump(x_temp, 'cleaned_train.joblib')

['cleaned_train.joblib']

In [18]:
y_temp = y[args[0:36_500]]
print(y_temp.shape)

(36500, 182)


In [19]:
dump(y_temp, 'cleaned_labels.joblib')

['cleaned_labels.joblib']

In [20]:
# t = np.sum(y_temp, axis = 1)
# np.where(t>1)[0].shape